In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Load dataset
df = pd.read_csv('german_credit_data.csv')

# Drop index column if it exists
if 'Unnamed: 0' in df.columns:
    df.drop('Unnamed: 0', axis=1, inplace=True)

# Create synthetic target without leaking the label
df['Risk'] = df['Credit amount'].apply(lambda x: 'Bad' if x > 5000 else 'Good')

# Encode categorical variables
le = LabelEncoder()
for col in ['Sex', 'Housing', 'Saving accounts', 'Checking account', 'Purpose']:
    df[col] = le.fit_transform(df[col].astype(str))

# Define features (exclude 'Credit amount' to prevent leakage)
X = df.drop(['Risk', 'Credit amount'], axis=1)
y = le.fit_transform(df['Risk'])  # 'Good'=1, 'Bad'=0

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Logistic Regression
lr = LogisticRegression()
lr.fit(X_train_scaled, y_train)
y_pred_lr = lr.predict(X_test_scaled)

# Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

# Evaluate
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_lr))
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))

print("\nLogistic Regression Report:\n", classification_report(y_test, y_pred_lr))
print("\nRandom Forest Report:\n", classification_report(y_test, y_pred_rf))


Logistic Regression Accuracy: 0.905
Random Forest Accuracy: 0.905

Logistic Regression Report:
               precision    recall  f1-score   support

           0       0.76      0.53      0.63        30
           1       0.92      0.97      0.95       170

    accuracy                           0.91       200
   macro avg       0.84      0.75      0.79       200
weighted avg       0.90      0.91      0.90       200


Random Forest Report:
               precision    recall  f1-score   support

           0       0.70      0.63      0.67        30
           1       0.94      0.95      0.94       170

    accuracy                           0.91       200
   macro avg       0.82      0.79      0.81       200
weighted avg       0.90      0.91      0.90       200

